需求:

北京明天的天气怎么样? 上海呢? (通过两次对话实现)

以ReAct模式为例进行实现:

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.tools import StructuredTool
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

dotenv.load_dotenv()
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 获取Tavily搜索工具的实例
search = TavilySearchResults(max_results=3)


# 方式2: 使用Tool
search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm =ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0,
)

# 获取一个记忆的实例: ConversationBufferMemory
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history"  # 此时必须设置memory_key, 且值为chat_history, 因为提示词模板中使用的是chat_history, 二者必须一致!
)

# 获取Agent的实例
agent = AgentType.CONVERSATIONAL_REACT_DESCRIPTION



# 获取AgentExecutor的实例
agent_executor = initialize_agent(
    tools = [search_tool],
    llm = llm,
    agent = agent,
    verbose = True, # 显示详细的日志信息
    memory = memory,
)

# 通过AgentExecutor 调用invoke()，并得到响应
result = agent_executor.invoke("查询北京今天的天气情况")

# 处理响应数据
print(result)

In [ ]:
result = agent_executor.invoke("上海呢？")

# 处理响应数据
print(result)